In [ ]:
#|default_exp concurrent.remote.test_ProxyPort_0

In [ ]:
#|export
import asyncio, os, sys, subprocess
from multiprocessing.connection import Listener
from fbdev._utils import TaskManager, find_available_port
from fbdev.dev_utils import is_in_repl
from fbdev.comp.port import PortType, PortSpec, Port
from fbdev.comp.packet import Packet
from fbdev.concurrent.remote import RemoteController, get_client, ProxyPort, ProxyPortMediator

In [ ]:
#|export
port_num = find_available_port()

remote_py_script = """
address = ('localhost', %s)
authkey = b'secret'

import asyncio, os, multiprocessing
from multiprocessing.connection import Listener
from fbdev._utils import TaskManager, StateHandler
from fbdev.comp.port import PortType, PortSpec, Port
from fbdev.concurrent.remote import RemoteController, ProxyPort, ProxyPortMediator

listener = Listener(address, authkey=authkey)
async def main(conn):
    task_manager = TaskManager(None)
    remote = RemoteController(conn, task_manager)
    
    port_spec = PortSpec(PortType.INPUT, 'in')
    port = Port(port_spec)
    proxy_port_mediator = ProxyPortMediator('main', remote, task_manager, port)
    
    remote.send_ready()
    await remote.await_ready()
    
    data = await port._get_and_consume()
    assert data == 'Hello from parent!'
    print("From parent:", data)
    
    await port._put_value('Hello from child!')
    
    await asyncio.sleep(1)
    
with listener.accept() as conn:
    asyncio.run(main(conn))
""" % port_num

address = ('localhost', port_num)
authkey = b'secret'

async def test():
    task_manager = TaskManager(None)
    proc = subprocess.Popen([sys.executable, '-c', remote_py_script])

    with get_client(address, authkey=authkey) as conn:
        remote = RemoteController(conn, task_manager)
        
        port_spec = PortSpec(PortType.INPUT, 'in')
        proxy_port = ProxyPort('main', remote, task_manager, port_spec, None)
        
        remote.send_ready()
        await remote.await_ready()
        await proxy_port.await_initialised()
        
        await proxy_port.states.get_awaiting.wait(True)
        await proxy_port._put(Packet('Hello from parent!'))
        data = await proxy_port._get_and_consume()
        assert data == 'Hello from child!'
        print("From child:", data)

        await asyncio.sleep(1)
        
    await task_manager.destroy()
    proc.communicate()  # This will wait until the process completes

In [ ]:
await test()

From parent: Hello from parent!
From child: Hello from child!
